In [1]:
import preprocess
import Model
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
import numpy as np
from keras.optimizers import adam_v2
import tensorflow as tf


config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9 # 占用GPU90%的显存
sess=tf.compat.v1.Session(config=config)

batch_size =64
epochs = 20
num_classes = 3
length = 5120
BatchNorm = True # 是否批量归一化
number = 200# 每类样本的数量
normal = True # 是否标准化
rate = [0.7,0.2,0.1] # 测试集验证集划分比例

def data_pre(path,number):
    x_train, y_train, x_valid, y_valid, x_test, y_test = preprocess.prepro(d_path=path, length=length,
                                                                           number=number,
                                                                           normal=normal,
                                                                           rate=rate,
                                                                           enc=False, enc_step=28)
    x_train = x_train.astype('float32')
    y_train = y_train.astype('float32')
    x_valid = x_valid.astype('float32')
    y_valid = y_valid.astype('float32')
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    x_train, x_valid, x_test = x_train[:, :, np.newaxis], x_valid[:, :,np.newaxis], x_test[:, :, np.newaxis]
    #x_train, x_valid, x_test = x_train[:, :, np.newaxis, np.newaxis], x_valid[:, :, np.newaxis, np.newaxis], x_test[:, :, np.newaxis, np.newaxis]
    return x_train,x_valid,x_test,y_train,y_valid,y_test

In [3]:

if __name__ == '__main__':
    path = r".\Train"
    path_test = r".\Test"
    x_train,x_valid,x_test,y_train,y_valid,y_test = data_pre(path,200)
    x_train1,x_valid1,x_test1,y_train1,y_valid1,y_test1 = data_pre(path_test,200)
    # 输入数据的维度
    input_shape =x_train.shape[1:]
    print('训练样本维度:', x_train.shape)
    print(x_train.shape[0], '训练样本个数')
    print('验证样本的维度', x_valid.shape)
    print(x_valid.shape[0], '验证样本个数')
    print('测试样本的维度', x_test.shape)
    print(x_test.shape[0], '测试样本个数')

    model = Model.CFSPT(input_shape)
    #model.summary()
    # 定义优化器
    #Nadam1 = Nadam(lr=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-08,schedule_decay=0.004)
    adam = adam_v2.Adam(learning_rate=0.001)
    # 定义优化器，loss function, 训练过程中计算准确率
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])
    # 画出网络结构
    # plot_model(model, to_file='model_cnn.png', show_shapes=True, show_layer_names='False', rankdir='TB')
    callback_list = [ModelCheckpoint(filepath='CFSPT.hdf5', verbose=1, save_best_only=True,monitor='val_acc',mode='auto')]
    # 训练模型
    model.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=epochs,
              verbose=1, validation_data=(x_valid, y_valid), shuffle=True,
              callbacks=callback_list)

    model.load_weights('CFSPT.hdf5')
    # 评估模型
    #model.summary()
    loss, acc = model.evaluate(x_train1, y_train1)
    print("loss:", loss)
    print("accuracy", acc)



训练样本维度: (8400, 5120, 1)
8400 训练样本个数
验证样本的维度 (2400, 5120, 1)
2400 验证样本个数
测试样本的维度 (1200, 5120, 1)
1200 测试样本个数
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 5120, 1)]    0                                            
__________________________________________________________________________________________________
patch_encoder_cum_1 (PatchEncod (None, 19, 64)       34048       input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 19, 64)       12352       patch_encoder_cum_1[0][0]        
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 19, 64)       12352       patch_encoder_cum_

Epoch 1/20
132/132 [==============================] - 8s 36ms/step - loss: 0.4151 - acc: 0.8245 - val_loss: 0.1517 - val_acc: 0.9171

Epoch 00001: val_acc improved from -inf to 0.91708, saving model to CFSPT.hdf5
Epoch 2/20
  1/132 [..............................] - ETA: 4s - loss: 0.1148 - acc: 0.9531

D:\Coder\Python3.7\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


132/132 [==============================] - 4s 31ms/step - loss: 0.1124 - acc: 0.9456 - val_loss: 0.1983 - val_acc: 0.9075

Epoch 00002: val_acc did not improve from 0.91708
Epoch 3/20
132/132 [==============================] - 4s 30ms/step - loss: 0.0513 - acc: 0.9819 - val_loss: 0.0238 - val_acc: 0.9937

Epoch 00003: val_acc improved from 0.91708 to 0.99375, saving model to CFSPT.hdf5
Epoch 4/20
132/132 [==============================] - 4s 30ms/step - loss: 0.0193 - acc: 0.9940 - val_loss: 0.0203 - val_acc: 0.9921

Epoch 00004: val_acc did not improve from 0.99375
Epoch 5/20
132/132 [==============================] - 4s 30ms/step - loss: 0.0219 - acc: 0.9921 - val_loss: 0.0263 - val_acc: 0.9933

Epoch 00005: val_acc did not improve from 0.99375
Epoch 6/20
132/132 [==============================] - 4s 31ms/step - loss: 0.0159 - acc: 0.9952 - val_loss: 0.0073 - val_acc: 0.9975

Epoch 00006: val_acc improved from 0.99375 to 0.99750, saving model to CFSPT.hdf5
Epoch 7/20
132/132 [=======